# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [2]:
!pip install pyorient

     -------------------------------------- 69.0/69.0 kB 469.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyorient: filename=pyorient-1.5.5-py3-none-any.whl size=79037 sha256=68451a35dcc96ff0acfea380c391670a5f02e1fd4ffa0e0d561e2f8ecb2a874e
  Stored in directory: c:\users\fkpamegan\appdata\local\pip\cache\wheels\b9\34\12\9c3b860fddae6212a2677cd6cc4dd25ead9943b80296d22098
Successfully built pyorient


In [3]:
import pyorient

In [4]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [5]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [ ]:
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [9]:
# Connect your pyorient client to the `gods` database.
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [11]:
# Create a new class called `God`.
client.command("create class God extends V")

[11]

In [12]:
# Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`
zeus = client.command("CREATE VERTEX God SET name = 'Zeus', symbol = 'thunder'")[0]

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [15]:
# Print all current vertices in `gods`
print(client.command("SELECT FROM God"))


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [16]:
hera = client.command("CREATE VERTEX God SET name = 'Héra', symbol = 'tiara'")[0]
poseidon = client.command("CREATE VERTEX God SET name = 'Poséidon', symbol = 'trident'")[0]
athena = client.command("CREATE VERTEX God SET name = 'Athena', symbol = 'helmet'")[0]
ares = client.command("CREATE VERTEX God SET name = 'Arès', symbol = 'weapons'")[0]

**Q:** Display all vertices with name = `Arès`

In [18]:
# Display all vertices with name = `Arès`
print(client.command("SELECT FROM God WHERE name = 'Arès'"))

**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [19]:
# Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Poséidon') SET kind = 'sibling'")

**Q:** Redisplay all vertices, discuss.

In [20]:
print(client.command("SELECT FROM God"))


[<pyorient.otypes.OrientRecord object at 0x000001F9C107A608>, <pyorient.otypes.OrientRecord object at 0x000001F9C1074208>, <pyorient.otypes.OrientRecord object at 0x000001F9C1074148>, <pyorient.otypes.OrientRecord object at 0x000001F9C1074588>, <pyorient.otypes.OrientRecord object at 0x000001F9C1074748>]


**Q:** Display all edges. They are contained in the class `E`

In [21]:
# Display all edges. They are contained in the class `E`
print(client.command("SELECT FROM E"))

Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [22]:
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Héra')")
client.command("CREATE EDGE Father FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Arès')")
client.command("CREATE EDGE Father FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Athena')")
client.command("CREATE EDGE Mother FROM (SELECT FROM God WHERE name = 'Héra') TO (SELECT FROM God WHERE name = 'Arès')")
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Héra') TO (SELECT FROM God WHERE name = 'Zeus')")
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Poséidon') TO (SELECT FROM God WHERE name = 'Zeus')")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [23]:
print(client.command("SELECT expand(out()) FROM God WHERE name = 'Zeus'"))

[<pyorient.otypes.OrientRecord object at 0x000001F9C104CDC8>, <pyorient.otypes.OrientRecord object at 0x000001F9C1098FC8>, <pyorient.otypes.OrientRecord object at 0x000001F9C1090288>, <pyorient.otypes.OrientRecord object at 0x000001F9C1090508>]


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [24]:
# Display all vertices which got a father
print(client.command("SELECT expand(in('Father')) FROM God"))

[<pyorient.otypes.OrientRecord object at 0x000001F9C105F108>, <pyorient.otypes.OrientRecord object at 0x000001F9C1089648>]


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [25]:
# Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.
print(client.command("SELECT expand(out('Mother')) FROM God"))

**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [26]:
# Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).
print(client.command("SELECT expand(out('Sibling')) FROM God WHERE name = 'Zeus'"))

[<pyorient.otypes.OrientRecord object at 0x000001F9C1089B48>, <pyorient.otypes.OrientRecord object at 0x000001F9C1089FC8>]


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [27]:
# create eCommerce db
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

In [28]:
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

{{'name': 'E'},'version':0,'rid':'#-2:0'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:1'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:2'}
{{'name': 'ORole'},'version':0,'rid':'#-2:3'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:4'}
{{'name': 'V'},'version':0,'rid':'#-2:5'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:6'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:7'}
{{'name': 'OTriggered'},'version':0,'rid':'#-2:8'}
{{'name': 'OUser'},'version':0,'rid':'#-2:9'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [29]:
# Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.
client.command("CREATE CLASS Product extends V")
client.command("CREATE VERTEX Product SET name = 'spaghetti'")
client.command("CREATE VERTEX Product SET name = 'bolognese sauce'")
client.command("CREATE VERTEX Product SET name = 'cheese'")
client.command("CREATE VERTEX Product SET name = 'apple'")

**Q:** Create the following people: `peter`, `meredith`.

In [30]:
# Create the following people: `peter`, `meredith`.
client.command("CREATE CLASS Person extends V")
client.command("CREATE VERTEX Person SET name = 'peter'")
client.command("CREATE VERTEX Person SET name = 'meredith'")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [31]:

# - peter > spaghetti + cheese on 20/01/2016
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'spaghetti') SET date = '20/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'cheese') SET date = '20/01/2016'")
# - meredith > cheese + apple + bolognese sauce on 22/01/2016
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'meredith') TO (SELECT FROM Product WHERE name = 'cheese') SET date = '22/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'meredith') TO (SELECT FROM Product WHERE name = 'apple') SET date = '22/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'meredith') TO (SELECT FROM Product WHERE name = 'bolognese sauce') SET date = '22/01/2016'")
# - peter > spaghetti + bolognese sauce on 27/01/2016
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'spaghetti') SET date = '27/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'bolognese sauce') SET date = '27/01/2016'")

**Q:** Who bought Bolognese sauce?

In [32]:
print(client.command("SELECT expand(in('Bought')) FROM Product WHERE name = 'bolognese sauce'"))

[<pyorient.otypes.OrientRecord object at 0x000001F9C1088188>, <pyorient.otypes.OrientRecord object at 0x000001F9C1089F08>]


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [33]:
#  What products are purchased with Bolognese sauce? link in and out
print(client.command("SELECT expand(out('Bought')) FROM (SELECT FROM Product WHERE name = 'bolognese sauce')"))



[]


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.